# National Vulnerability Database Download

## Imports

In [1]:
from datetime import datetime, timedelta
import requests
import time
import pandas as pd
import os

## Format URL for Request

In [2]:
def formatURL(modEndDate):
    
    """
    pass modEndDate as datetime and return formatted URL
    """
    
    # set constants
    base_url = "https://services.nvd.nist.gov/rest/json/cves/1.0"
    apiKey = "06cc14f7-5eef-4379-9210-1ab6ddf8c234"
    resultsPerPage = 2000 
    
    # modStartDate is one day before modEndDate
    modStartDate = (modEndDate - timedelta(days = 1)).strftime("%Y-%m-%dT00:00:00:000 UTC")
    modEndDate = modEndDate.strftime("%Y-%m-%dT00:00:00:000 UTC")

    # format URL
    url = f"{base_url}?apiKey={apiKey}&modStartDate={modStartDate}&modEndDate={modEndDate}&resultsPerPage={resultsPerPage}"

    return url

## Determine what to parse

import json
json_string = '{"cve": {"data_type": "CVE", "data_format": "MITRE", "data_version": "4.0", "CVE_data_meta": {"ID": "CVE-2021-34357", "ASSIGNER": "security@qnap.com", "publishedDate": "2021-11-13", "lastModifiedDate": "2021-11-13"}, "problemtype": {"problemtype_data": [{"description": [{"lang": "en", "value": "CWE-79"}]}]}, "references": {"reference_data": [{"url": "https://www.qnap.com/en/security-advisory/qsa-21-47", "name": "https://www.qnap.com/en/security-advisory/qsa-21-47", "refsource": "MISC", "tags": []}]}, "description": {"description_data": [{"lang": "en", "value": "A cross-site scripting (XSS) vulnerability has been reported to affect QNAP device running QmailAgent. If exploited, this vulnerability allows remote attackers to inject malicious code. We have already fixed this vulnerability in the following versions of QmailAgent: QmailAgent 3.0.2 ( 2021/08/25 ) and later"}]}}, "configurations": {"CVE_data_version": "4.0", "nodes": []}, "impact": {}, "publishedDate": "2021-11-13T01:15Z", "lastModifiedDate": "2021-11-13T01:15Z"}'
json_obj = json.loads(json_string)

json_obj['cve']['problemtype']

## Parse CVEs

In [3]:
with open('cwes.csv', 'w') as f:
    f.write("CVE_ID, CWE_ID\n")

In [4]:
def flattenCVE(cve):
    
    """
    pull out CVE ID, cvss vector info
    """
    
    data = cve["cve"]["CVE_data_meta"]
    data['publishedDate'] = datetime.strptime(cve["publishedDate"], "%Y-%m-%dT%H:%MZ").strftime('%Y-%m-%d')
    data['lastModifiedDate'] = datetime.strptime(cve["lastModifiedDate"], "%Y-%m-%dT%H:%MZ").strftime('%Y-%m-%d')
    
    # extract cwe info (if available)
    try:
        
        with open('cwes.csv', 'a') as f:
            cwes = cve['cve']['problemtype']['problemtype_data'][0]['description']
            for cwe in cwes:
                f.write(f"{data['ID']}, {cwe['value']}\n")
                
    except Exception as error:
        pass
    
    # extract cvss v2 info (if available)
    try:
        
        cvssv2 = cve["impact"]["baseMetricV2"]["cvssV2"]
        cvssv2 = {"v2_" + str(key): val for key, val in cvssv2.items()}
        data.update(cvssv2)
        
    except Exception as error:
        pass
        
    # extract cvss v3 info (if available)
    try:
        
        cvssv3 = cve["impact"]["baseMetricV3"]["cvssV3"]
        cvssv3 = {"v3_" + str(key): val for key, val in cvssv3.items()}
        data.update(cvssv3)
    
    except Exception as error:
        pass
    
    # create and return pandas dataframe
    return pd.DataFrame(data.items()).set_index(0).T

## Iterate over Dates

In [5]:
def main(startDate):
    
    """
    Main Process
    """
    
    # create empty pandas dataframe
    df2 = pd.DataFrame()

    t1 = datetime.now()
    print (f"Starting at {t1.strftime('%H:%M:%S')}")

    # advance date until today
    while startDate < datetime.today():
    
        print (f"\rNow processing: {startDate.strftime('%Y-%m-%d')}", end="")
    
        # format URL
        url = formatURL(startDate)
    
        # get results for date
        result = requests.get(url).json()['result']['CVE_Items']
    
        # flatten result
        for item in result:
        
            row = flattenCVE(item)
            df2 = df2.append(row)
    
        time.sleep(5)
    
        startDate += timedelta(days = 1)

    t2 = datetime.now()
    print (f"\nEnded at {t2.strftime('%H:%M:%S')}")
    print (f"Elapsed time: {int((t2-t1).seconds/60)}:{(t2-t1).seconds%60}")
    
    return df2

## Write results to file(s)

In [6]:
filename = "cves.csv"

In [7]:
if os.path.exists(filename):
     
    # Set startDate to be 3 days before file last modified date
    lastmoddate = (datetime.fromtimestamp(os.path.getmtime(filename))-timedelta(days=3)).strftime("%Y-%m-%d")
    startDate = datetime.strptime(lastmoddate, "%Y-%m-%d")

    with open(filename, "rt") as file:
        df1 = pd.read_csv(file) 
    
    df2 = main(startDate)

    # Eliminate all but the last duplicate; Merge existing file with new data
    df3 = pd.concat([df1, df2])
    df3.drop_duplicates(subset=["ID"], keep="last", inplace=True)
    df3.rename(columns={'ID': 'CVE_ID'}, inplace=True)
    df3.to_csv(filename, index=False)
    
else:
    
    # Start date equals way back
    #startDate = datetime.strptime("2000-01-01", "%Y-%m-%d")
    startDate = datetime.strptime("2015-01-01", "%Y-%m-%d")
    
    df2 = main(startDate)

    # Eliminate all but the last duplicate; Create new file
    df2.drop_duplicates(subset=["ID"], keep="last", inplace=True)
    df2.rename(columns={'ID': 'CVE_ID'}, inplace=True)
    df2.to_csv(filename, index=False)

Starting at 21:47:02
Now processing: 2021-11-15
Ended at 03:56:10
Elapsed time: 369:7


## Automatically upload file to Google Drive

%pip install pydrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

gfile = drive.CreateFile({'parents': [{'id': '1b69zGZ7eTGufBdwP7NezLBeTpt0Xs0j1'}]})
gfile.SetContentFile(filename)
gfile.Upload()